# grab IMERG fields for the bounding box +/- 5 degrees and write out 


In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob 

import geopandas as gp

from shapely.geometry import Point
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

import warnings
warnings.filterwarnings('ignore')

## Read all lakes in 2014-2018 
#### lakes dataframe was "improved" in LakeCaseStudy.ipynb with firsttime added

In [2]:
df = pd.read_csv('ccvls_stats_2014-2018.improved.csv')
# df

In [3]:
equatorcases = df[ abs(df.coastlat) < 10 ] # 162 of them exceeding 1 day 
eq7cases = equatorcases[ equatorcases.dur_days > 6 ]  # 39 of them exceeding a week (more than 6 in days) 
eq7cases.maxarea.size

39

# Grab IMERG for the event bounding box

In [15]:
IMERG = xr.open_dataset('https://gpm1.gesdisc.eosdis.nasa.gov:443/dods/GPM_3IMERGHH_06')
IMERG

<xarray.Dataset>
Dimensions:   (time: 374016, lat: 1800, lon: 3600)
Coordinates:
  * time      (time) datetime64[ns] 2000-06-01 ... 2021-09-30T23:30:00
  * lat       (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon       (lon) float64 -179.9 -179.8 -179.8 -179.7 ... 179.8 179.8 179.9
Data variables:
    hqprecip  (time, lat, lon) float32 ...
    precipc   (time, lat, lon) float32 ...
Attributes:
    title:        GPM IMERG Final Precipitation L3 Half Hourly 0.1 by 0.1 deg...
    Conventions:  COARDS\nGrADS
    dataType:     Grid
    history:      Wed Oct 18 22:18:27 GMT 2023 : imported by GrADS Data Serve...

In [16]:
pchourly = IMERG.precipc[1::2,:,:]
pchourly

<xarray.DataArray 'precipc' (time: 187008, lat: 1800, lon: 3600)>
[1211811840000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01T00:30:00 ... 2021-09-30T23:30:00
  * lat      (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float64 -179.9 -179.8 -179.8 -179.7 ... 179.8 179.8 179.9
Attributes:
    long_name:  precipitationcal [mm/hr]

In [17]:
pc3hourly = IMERG.precipc[0::6,:,:]
pc3hourly

<xarray.DataArray 'precipc' (time: 62336, lat: 1800, lon: 3600)>
[403937280000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 ... 2021-09-30T21:00:00
  * lat      (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float64 -179.9 -179.8 -179.8 -179.7 ... 179.8 179.8 179.9
Attributes:
    long_name:  precipitationcal [mm/hr]

In [9]:
!date

Wed Oct 18 18:19:51 EDT 2023


# 3 hourly, for speed 

In [8]:
# 3 hourly 
for icase in range(len(eq7cases)): 

# Continue from where it last crashed/ended:
    if(icase < 24): 
        continue

# Select cases by small to large areatime
    case = eq7cases.sort_values('areatime').iloc[icase]
    filename = case.filename
    print(icase, ' case, areatime ',filename, case.areatime)

    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = pc3hourly.sel(lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                time=slice(gdf.time.min(),gdf.time.max())             )

    d2.to_netcdf('~/Box/VaporLakes/data/LAKEBYLAKE/IMERG/'+filename[0:-8]+'_IMERG_precipc_3h.nc')
    

0  case, areatime  2018_11_20_11_lat4p616S.geojson 1892.647086291892
1  case, areatime  2018_11_16_21_lat8p412S.geojson 2667.6861528738714
2  case, areatime  2014_04_12_21_lat5p095N.geojson 5008.481114870546
3  case, areatime  2018_10_22_15_lat9p781N.geojson 5261.462676095129
4  case, areatime  2017_05_28_11_lat5p291S.geojson 5690.601727745032
5  case, areatime  2014_05_11_15_lat4p404N.geojson 6050.818575241469
6  case, areatime  2014_05_11_18_lat2p131S.geojson 6060.850142585366
7  case, areatime  2017_12_01_11_lat9p845S.geojson 6534.053998839994
8  case, areatime  2017_04_29_11_lat7p001S.geojson 7493.2953775304195
9  case, areatime  2015_03_21_09_lat5p130N.geojson 8302.245634879724
10  case, areatime  2015_03_21_17_lat5p987N.geojson 8398.895739251338
11  case, areatime  2017_05_06_18_lat1p979S.geojson 8419.365461090125
12  case, areatime  2017_05_06_20_lat3p790S.geojson 8426.735231012779
13  case, areatime  2017_05_06_20_lat0p918N.geojson 8629.762051834938
14  case, areatime  2017_04_

Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


15  case, areatime  2016_04_10_23_lat8p819S.geojson 9290.8194783981
16  case, areatime  2017_05_08_05_lat1p160N.geojson 9695.01861094522
17  case, areatime  2016_04_29_17_lat1p821N.geojson 9754.70437103062
18  case, areatime  2016_05_01_00_lat6p733S.geojson 10522.284848804758
19  case, areatime  2016_05_02_05_lat1p551S.geojson 12023.310669858418
20  case, areatime  2014_04_17_06_lat6p223S.geojson 12712.592277747774


Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


21  case, areatime  2017_01_07_14_lat6p991S.geojson 12738.00772196208
22  case, areatime  2017_05_09_01_lat3p355S.geojson 13759.444602992908


...

Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


37  case, areatime  2015_05_06_21_lat5p111N.geojson 28508.157805491875


Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


38  case, areatime  2015_05_07_03_lat6p414S.geojson 29758.27803679196


Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


In [11]:
!date

Sun Jun 18 10:57:44 MDT 2023


# Hourly

In [19]:
# hourly, trying afresh in Miami office
# Only do the ones that don't exist 
import os.path

for icase in range(len(eq7cases)): 
# Select cases by small to large areatime
    case = eq7cases.sort_values('areatime').iloc[icase]
    filename = case.filename
    #print(icase, ' case, areatime ',filename, case.areatime)

    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = pchourly.sel(lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                time=slice(gdf.time.min(),gdf.time.max())             )

# delete the bad ones and do em again and again 
    fi = '~/Box/VaporLakes/data/LAKEBYLAKE/IMERG/'+filename[0:-8]+'_IMERG_precipc_hourly.nc'
    if not os.path.isfile(fi):
        print(icase, ' case NOT ALREADY THERE, areatime ',filename, case.areatime)
        d2.to_netcdf(fi)
    

0  case NOT ALREADY THERE, areatime  2018_11_20_11_lat4p616S.geojson 1892.647086291892
1  case NOT ALREADY THERE, areatime  2018_11_16_21_lat8p412S.geojson 2667.6861528738714
2  case NOT ALREADY THERE, areatime  2014_04_12_21_lat5p095N.geojson 5008.481114870546
3  case NOT ALREADY THERE, areatime  2018_10_22_15_lat9p781N.geojson 5261.462676095129
4  case NOT ALREADY THERE, areatime  2017_05_28_11_lat5p291S.geojson 5690.601727745032
5  case NOT ALREADY THERE, areatime  2014_05_11_15_lat4p404N.geojson 6050.818575241469
6  case NOT ALREADY THERE, areatime  2014_05_11_18_lat2p131S.geojson 6060.850142585366
7  case NOT ALREADY THERE, areatime  2017_12_01_11_lat9p845S.geojson 6534.053998839994
8  case NOT ALREADY THERE, areatime  2017_04_29_11_lat7p001S.geojson 7493.2953775304195
9  case NOT ALREADY THERE, areatime  2015_03_21_09_lat5p130N.geojson 8302.245634879724
10  case NOT ALREADY THERE, areatime  2015_03_21_17_lat5p987N.geojson 8398.895739251338
11  case NOT ALREADY THERE, areatime  20

In [26]:
!ls ~/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2015_05_07_03_lat6p414S_IMERG_precipc_hourly.nc

/Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2015_05_07_03_lat6p414S_IMERG_precipc_hourly.nc


In [23]:
os.path.isfile(fi)

False

In [12]:
# hourly, failed a lot in June on bad Boulder wifi 
for icase1 in range(len(eq7cases)): 
    icase = icase1+1
# Select cases by small to large areatime
    case = eq7cases.sort_values('areatime').iloc[icase]
    filename = case.filename
    print(icase, ' case, areatime ',filename, case.areatime)

    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = pchourly.sel(lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                time=slice(gdf.time.min(),gdf.time.max())             )

    d2.to_netcdf('~/Box/VaporLakes/data/LAKEBYLAKE/IMERG/'+filename[0:-8]+'_IMERG_precipc_hourly.nc')
    

0  case, areatime  2018_11_20_11_lat4p616S.geojson 1892.647086291892
1  case, areatime  2018_11_16_21_lat8p412S.geojson 2667.6861528738714


Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


2  case, areatime  2014_04_12_21_lat5p095N.geojson 5008.481114870546


Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


3  case, areatime  2018_10_22_15_lat9p781N.geojson 5261.462676095129


...

Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


37  case, areatime  2015_05_06_21_lat5p111N.geojson 28508.157805491875


Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


38  case, areatime  2015_05_07_03_lat6p414S.geojson 29758.27803679196


Error:curl error: Timeout was reached
Error:DAP DATADDS packet is apparently too short


In [13]:
!date

Mon Jun 19 06:28:15 MDT 2023


# Looks like all those timeouts were terrible 
## Max data values either 0 or 10^31

In [28]:
for file in glob('/Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly/*_hourly.nc'):
    ds = xr.open_dataset(file)
    print('Max from '+ file +' = ',ds.max().values() )


Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly/2014_04_12_21_lat5p095N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 120.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly/2014_05_11_18_lat2p131S_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 116.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly/2018_03_19_12_lat9p318S_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 120.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly/2018_10_22_15_lat9p781N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 98.82)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly/2015_05_01_21_lat7p053N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions

In [21]:
# an earlier run

for file in glob('/Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly_timedout/*_hourly.nc'):
    ds = xr.open_dataset(file)
    print('Max from '+ file +' = ',ds.max().values() )


Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly_timedout/2014_05_11_18_lat2p131S_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 116.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly_timedout/2018_10_22_15_lat9p781N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 98.82)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly_timedout/2015_05_01_21_lat7p053N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 0.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly_timedout/2017_04_29_11_lat0p732S_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 120.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/Hourly_timedout/2014_04_17_06_lat6p223S_IMERG_precipc_hourly.n

In [ ]:
ds

In [51]:

for file in glob('/Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/*_hourly.nc'):
    ds = xr.open_dataset(file)
    print('Max from '+ file +' = ',ds.max().values() )


Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2014_04_12_21_lat5p095N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 7.139e+31)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2014_05_11_18_lat2p131S_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 7.139e+31)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2018_03_19_12_lat9p318S_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 0.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2018_10_22_15_lat9p781N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 0.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2015_05_01_21_lat7p053N_IMERG_precipc_hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    preci

# 3hourly is better but still has problems

In [52]:

for file in glob('/Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/*3hourly.nc'):
    ds = xr.open_dataset(file)
    print('Max from '+ file +' = ',ds.max().values() )


Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2017_05_06_18_lat1p979S_IMERG_precipc_3hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 107.7)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2017_02_27_22_lat8p004S_IMERG_precipc_3hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 0.0)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2015_04_30_17_lat4p330S_IMERG_precipc_3hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 93.52)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2017_05_06_20_lat3p790S_IMERG_precipc_3hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precipc  float32 107.7)
Max from /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/IMERG/2016_05_02_05_lat1p551S_IMERG_precipc_3hourly.nc =  ValuesView(<xarray.Dataset>
Dimensions:  ()
Data variables:
    precip